In [20]:

import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt


failure_data = pd.read_csv(r'Failure_modelling\Failure_Data\failure_data.csv', index_col=0)
pipe_data = pd.read_csv(r'Failure_modelling\Failure_Data\pipe_data.csv', index_col=0)

In [36]:
ac_pipes = pipe_data[pipe_data.Materiaalcode == 'MAT_AC']
ac_failures = failure_data[failure_data.storing_materiaal == 'MAT_AC']
model_failures = ac_failures[['storing_materiaal', 'storing_diameter', 'storing_melddatum', 'storing_aanlegjaar', 'age_at_failure']]
model_failures.loc[:,'age_at_failure'] = model_failures.loc[:,'age_at_failure'].astype(int)
model_pipes = ac_pipes[['Aanlegjaar', 'Lengte', 'Diameter', 'Materiaalcode']]
print(model_pipes.head())
print(model_failures.head())


         Aanlegjaar      Lengte  Diameter Materiaalcode
2166783        1974  368.595185       200        MAT_AC
2110184        1990    0.500100       150        MAT_AC
2144900        1989   35.832798       200        MAT_AC
2104664        1979   15.248820       100        MAT_AC
2182521        1973  280.118530       100        MAT_AC
   storing_materiaal  storing_diameter    storing_melddatum  \
4             MAT_AC              50.0  2010-03-10 00:00:00   
7             MAT_AC             160.0  2010-11-19 00:00:00   
8             MAT_AC              50.0  2010-04-16 00:00:00   
9             MAT_AC             100.0  2010-11-15 00:00:00   
10            MAT_AC             100.0  2010-07-19 00:00:00   

   storing_aanlegjaar  age_at_failure  
4          1960-01-01            50.0  
7          1963-01-01            47.0  
8          1969-01-01            41.0  
9          1955-01-01            55.0  
10         1968-01-01            42.0  


In [29]:
model_pipes =  model_pipes[model_pipes['Aanlegjaar'] > 2024 - 40]
model_pipes

,Aanlegjaar,Lengte,Diameter,Materiaalcode
2110184,1990,0.500100,150,MAT_AC
2144900,1989,35.832798,200,MAT_AC
2110578,1991,53.837175,150,MAT_AC
2058528,1989,59.412268,100,MAT_AC
2294746,1987,200.502698,100,MAT_AC
...,...,...,...,...
2119131,1986,29.096184,100,MAT_AC
2235544,1990,10.228036,150,MAT_AC
2155760,1990,97.426508,100,MAT_AC
2060033,1988,65.207266,100,MAT_AC
